<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/06_pytorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Pytorch Tranfer Learning

O que eh Transfer Learning?

Transfer Learning envolve pegar os parametros de um modelo que foi trainado em outros dataset e aplicar-los no nosso proprio ploblema

* Pretrained model = foundation models

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)


2.4.1+cu121
0.19.1+cu121


In [2]:
#COpiado da estrutura do curso, modificado apenas para manter a pasta "data" ao baixar o repositorio para o collab
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .

    #!mv pytorch-deep-learning/data .

    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 4356 (delta 213), reused 254 (delta 178), pack-reused 4035 (from 1)
Receiving objects: 100% (4356/4356), 654.51 MiB | 31.50 MiB/s, done.
Resolving deltas: 100% (2585/2585), done.
Updating files: 100% (248/248), done.


In [3]:
#COnfigurando o dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# 1. Conseguindo os dados

Baixandoi os dados: Pizza, Steak, Sushi para construir um modelo com transfer learning

In [5]:
import os
import zipfile
import requests

from pathlib import Path



#Configurando o caminho
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

#Se a pasta das imagens nao existe, baixar e prepara
if image_path.is_dir():
  print(f"Diretrio {image_path} existe, pulando download...")
else:
  print(f"O diretorio {image_path} nao foi encontrado, baixando...")
  image_path.mkdir(parents=True, exist_ok=True)

  #Baixando dados -> pizza, steak, sushi
  with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Baixando pizza, steak, sushi...")
    f.write(request.content)

  # Unzip pizza, steak, sushi
  with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
      print("Unzipping pizza, steak, sushi data...")
      zip_ref.extractall(image_path)

  # Remove .zip
  os.remove(data_path / "pizza_steak_sushi.zip")


Diretrio data/pizza_steak_sushi existe, pulando download...


In [6]:
#Configurando caminho dos diretorios
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))